In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
# This file is a simple implementation of the 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
model = Sequential()
eurusd_train = pd.read_csv(r"/storage/eurusd_train_normed.csv")
eurusd_test = pd.read_csv(r"/storage/eurusd_test_normed.csv")
eurusd_train = eurusd_train.replace(np.nan, 0)
eurusd_test = eurusd_test.replace(np.nan, 0)

# TODO Friday-> Tomorrow
# Get LSTM model working with your train,test data first
# get it learning and giving output. Then look to refine the model. think about the target, how will we create probability distributions? 
# get the EURUSD data
# Chose the trainign size of the data
DATA_SIZE = 475
#npData.shape
train_original = eurusd_train.iloc[-DATA_SIZE:,:]
test_original = eurusd_test.iloc[:DATA_SIZE,:]
features_to_use = ["spot_v_HF", "spot_v_MF", "spot_v_LF", "HF_ema_diff", "MF_ema_diff","LF_ema_diff", "LDN", "NY", "Asia", "target"]
# Data needs to be reshaped into 3D tensor
train_sample = eurusd_train[features_to_use].iloc[-DATA_SIZE:,:].values
test_sample = eurusd_test[features_to_use].iloc[:DATA_SIZE,:].values
#sample_data['target'] = 1
#target.shape
# this creates your dataset with the lookback periods it needs to use.

import numpy as np
# this lookbacks over a set period as the memory for the LSTM
look_back = 60
def create_dataset(dataset, populate_target ,look_back=look_back):
  dataX, dataY, target_dates = [], [], []
  for i in range(len(dataset)-look_back+1):
      # this takes the very last col as the target
    a = dataset[i:(i+look_back), :-1]
    dataX.append(a)
    # this code assumes that the target vector is the very last col.
    dataY.append(dataset[i + look_back - 1, -1])
    if populate_target:
        target_dates.append(test_original['Date'].loc[i + look_back - 1])
  return np.array(dataX), np.array(dataY), target_dates

train_sample_transformed = create_dataset(train_sample,False, look_back)
train_data = train_sample_transformed[0]
train_target = train_sample_transformed[1]

test_sample_transformed = create_dataset(test_sample, True, look_back)
test_data = test_sample_transformed[0]
test_target = test_sample_transformed[1]
target_dates = test_sample_transformed[2]

# reshape sems to add another list around every observation
train_data = train_data.reshape(train_data.shape[0], look_back, train_data.shape[2])
train_target = train_target.reshape(train_target.shape[0], 1)

test_data = test_data.reshape(test_data.shape[0], look_back, test_data.shape[2])
test_target = test_target.reshape(test_target.shape[0], 1)

BATCH_SIZE = 300
no_features = train_data.shape[2]
model = Sequential()
model.add(LSTM(4,batch_input_shape = (None,look_back,no_features), return_sequences = True))
model.add(LSTM(1, return_sequences = False, activation="softmax"))
#model.add(Dense(2, activation= "softmax"))
model.compile(loss = "mean_absolute_error", optimizer="adam", metrics = ['accuracy'])
model.summary()


In [ ]:
history = model.fit(train_data,train_target,epochs = 1000,validation_data=(test_data,test_target), verbose=0)

In [ ]:
# mergeing signals to the df
predictions = pd.DataFrame({"Date" : target_dates,"Predictions": predicted})
test_original = pd.merge(test_original,predictions,how="left", on="Date").fillna(0)
test_original.columns

In [ ]:
trade_horizon = 24
test_original["scaled_signal"] = test_original['Predictions'].shift(2).rolling(trade_horizon).sum()/trade_horizon
# no shift needed as we have already done that inpervious step
test_original['strat_returns'] = test_original['logret']*test_original['scaled_signal']
test_original['strat_returns_sum'] = test_original['strat_returns'].cumsum()

In [ ]:
ax1 = plt.plot()
s1 = test_original['CCY']
plt.plot(s1,'b')
plt.ylabel('Ret',color='b')
ax2 = plt.gca().twinx()
s2 = test_original['strat_returns_sum'].loc[test_original['scaled_signal'] <0]
ax2.plot(s2, 'r*')
plt.ylabel('sin', color='r')
s3 = test_original['strat_returns_sum'].loc[test_original['scaled_signal'] >0]
ax2.plot(s3, 'g*')
plt.show()
test_original.to_csv(r"/storage/signal_prediction_test.csv",index = False)